# Extraction and classification of classical references from the ConDÉ corpus

Script written by Morgane Pica for a submission to the symposium ["Lire les classiques en Normandie"](https://rmblf.be/2022/02/04/appel-a-contribution-lire-les-classiques-en-normandie/) (oct 2022), to be written by herself and Mathieu Goux.

## Imports & declarations

In [35]:
from tqdm.notebook import tqdm #tqdm est bibliothèque qui permet d'avoir une barre de progression
import xml.etree.ElementTree as ET
import csv

ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
ET.register_namespace('xml','http://www.w3.org/XML/1998/namespace')

# Not all witnesses were enriched with reference identification.
witnesses = ["basnage","berault","merville","pesnelle","terrien"]
binpath = "/home/mpica/Progs/perso/CONDE/editions/base-version/"
einpath = "_base.xml"

listfile = "authors.csv"
tablefile = "mentions.csv"
authortable = "authors.csv"
checklist = "checklist.xml"

## FUNCTION: extract authors and store in a dictionary

In [45]:
authors = {}

def extract(witness, path):
    print(witness)
    
    """
    Function taking the name and path to a TEI-XML text file and
    analyzing the references, doing two things:
        - returning an XML element named after the current witness,
            itself containing the copy of each reference declaration,
        - completing the general author dictionnary with new elements,
            whether new authors or authors whose informations were
            incomplete.
            
    :param witness: Name (=id) of the current witness as str (no space)
    :param path: Path to the current witness TEI-XML file.
    
    """
    
    # Create the witness element.
    liste = ET.Element(witness)
    
    # Open and parse TEI-XML file.
    with open(path) as filein:
        tree = ET.parse(filein)
        root = tree.getroot()
        
        # Look for all declared authors.
        for author in tqdm(root.findall('.//{http://www.tei-c.org/ns/1.0}listPerson/{http://www.tei-c.org/ns/1.0}person')):
            
            # Get <birth> element.
            fullbirth = author.find('.//{http://www.tei-c.org/ns/1.0}birth')
            fulldeath = author.find('.//{http://www.tei-c.org/ns/1.0}death')
            
            try:
                # Get current author identifier.
                ident = author.get("{http://www.w3.org/XML/1998/namespace}id")
                
                if ident not in authors.keys():
                    # Create a dict. entry for the current author.
                    authors[ident] = {}
            
                    try:
                        # Get current author birth date.
                        
                        if "when" in fullbirth.attrib.keys():
                            authors[ident]["earliest-birth"] = fullbirth.get("when")
                            authors[ident]["latest-birth"] = fullbirth.get("when")
                        else:
                            authors[ident]["earliest-birth"] = fullbirth.get("notBefore")
                            authors[ident]["latest-birth"] = fullbirth.get("notAfter")
                            
                    except:
                        authors[ident]["earliest-birth"] = "none"
                        authors[ident]["latest-birth"] = "none"
                        liste.append(author)
                    
                    try:
                        # Get current author death date.
                        
                        if "when" in fullbirth.attrib.keys():
                            authors[ident]["earliest-death"] = fulldeath.get("when")
                            authors[ident]["latest-death"] = fulldeath.get("when")
                        else:
                            authors[ident]["earliest-death"] = fulldeath.get("notBefore")
                            authors[ident]["latest-death"] = fulldeath.get("notAfter")
                            
                    except:
                        authors[ident]["earliest-death"] = "none"
                        authors[ident]["latest-death"] = "none"
                        liste.append(author)
                        

                    try:
                        # Create a dict. to store all recorded names for current author.
                        lg = {}
                        
                        # Loop on names, store their language.
                        for name in author.findall('.//{http://www.tei-c.org/ns/1.0}persName'):
                            namelang = name.get("{http://www.w3.org/XML/1998/namespace}lang")
                            
                            if name.text:
                                # If name is not split into <forename>/<surname> elements,
                                # there is text directly into <persName> element and we
                                # make this the current language text.
                                lg[namelang] = name.text
                                
                            else:
                                # If name is split, the order is unsure, therefore
                                # we store each kind into its own entry
                                # within names dict. and make a final str out of it.
                                names = {}
                                for nchild in name.findall('*'):
                                    if nchild.tag == "{http://www.tei-c.org/ns/1.0}forename":
                                        names["fn"] = nchild.text
                                    elif nchild.tag == "{http://www.tei-c.org/ns/1.0}surname":
                                        names["sn"] = nchild.text
                                    
                                lg[namelang] = names["fn"] + " " + names["sn"]
                                
                        
                        # Setting an order of preference for final display of name:
                        # preferably French, if not, Latin, and if neither, English.
                        # (These are the only three name languages within the corpus.)
                        if "fr" in lg.keys():
                            authors[ident]["name"] = lg["fr"]
                        elif "la" in lg.keys():
                            authors[ident]["name"] = lg["la"]
                        elif "eng" in lg.keys():
                            authors[ident]["name"] = lg["eng"]
                            
                    except:
                        authors[ident]["name"] = "none"
                        liste.append(author)
                    
                # If the author was recorded in a previous witness but has no name,
                # we try to make a name string again with this witness.
                
                elif authors[ident]["name"] == "none":
                    
                    try:
                        lg = {}
                        for name in author.findall('.//{http://www.tei-c.org/ns/1.0}persName'):
                            namelang = name.get("{http://www.w3.org/XML/1998/namespace}lang")
                            if name.text:
                                lg[namelang] = name.text
                                
                            else:
                                names = {}
                                for nchild in name.findall('*'):
                                    if nchild.tag == "{http://www.tei-c.org/ns/1.0}forename":
                                        names["fn"] = nchild.text
                                    elif nchild.tag == "{http://www.tei-c.org/ns/1.0}surname":
                                        names["sn"] = nchild.text
                                        
                                lg[namelang] = names["sn"] + ", " + names["fn"]
                                                            
                        if "fr" in lg.keys():
                            authors[ident]["name"] = lg["fr"]
                        elif "la" in lg.keys():
                            authors[ident]["name"] = lg["la"]
                        elif "eng" in lg.keys():
                            authors[ident]["name"] = lg["eng"]
                            
                    except:
                        authors[ident]["name"] = "none"
                        liste.append(author)
                    
            except:
                continue
    
    return liste

## FUNCTION: sort extracted authors: keep those born before year 550

In [37]:
def sort(dico):
    
    """
    Function taking a dictionary of authors shaped like so:
    {'authorID': {'birth':'0000', 'name':'AuthorName'}}
    and returning the same dictionnary where all authors with a
    number greater than 550 as a birthdate were removed.
    
    :param dico: dict
    
    """
    print("Now sorting authors.")
    # Looping on author identifiers (=keys of dict.)
    for author in tqdm(dico.keys()):
        
        birth = authors[author]["latest-birth"]
        
        try:
            # If the author was born before 550, the entry is added
            # to the new dictionary.
            
            if int(birth) < 550:
                final[author] = authors[author]
        except:
            
            # If there is an error (no birth date), print the author
            # as we want to know if they are interesting now
            # (if so, we can correct the XML itself).
            
            print(authors[author])
    
    # Show me the final dictionary to assess the data.
    print(final)
    
    return final

## FUNCTION: get locations of all references

In [38]:
def get_refs(witness, path, authors):
    print(witness)
    """
    Function taking the name and path to a witness, as well
    as a list of author identifiers previously extracted, and
    fills the mentions dict. with identification information
    about each mention of each author in the list.
    
    
    :param authors: list of xml identifiers.
    """
    
    partcount = 0
    chptcount = 0
    sctcount = 0
    
    # Open and parse TEI-XML file.
    with open(path) as filein:
        tree = ET.parse(filein)
        root = tree.getroot()
        
        # Enter each part, chapter and section, keeping track of
        # their respective numbers. Chapter and section counts do not
        # start anew with each new parent, so as to have a unique number
        # within the document.
        for part in tqdm(root.findall('.//{http://www.tei-c.org/ns/1.0}div[@type="part"]')):
            partcount += 1
            
            for chapter in part.findall('.//{http://www.tei-c.org/ns/1.0}div[@type="chapter"]'):
                chptcount += 1
                
                for section in chapter.findall('.//{http://www.tei-c.org/ns/1.0}div[@type="section"]'):
                    sctcount += 1
                    
                    for ref in section.findall('.//{http://www.tei-c.org/ns/1.0}ref'):
                        
                        # Only work if the mention as an @corresp (there are others and
                        # these are of no interest here).
                        
                        if ref.get('corresp'):
                            ident = ref.get('corresp').replace("#","")
                            
                            if ident in authors and ident in mentions:
                                mentions[ident] += [[witness, str(partcount), str(chptcount), str(sctcount)]]
                            
                            elif ident in authors:
                                mentions[ident] = [[witness, str(partcount), str(chptcount), str(sctcount)]]

## FUNCTION: make a CSV file out of all this

In [39]:
def initial_csv(mentions, info):
    
    """
    Function writing the final CSV compiling author information
    and author mentions.
    
    :param mentions: a dictionary with author identifiers as keys
        (and a list of his mentions within the corpus as value)
        
    :param info: a dictionary with author identifiers as keys
        (and a dictionary of his personal information as value)
    """
    
    # Columns for the new CSV file.
    
    columns = [
        "Author",
        "Birth start",
        "Birth stop",
        "Death start",
        "Death stop",
        "Witness",
        "Part",
        "Chapter",
        "Section"
    ]
    
    
    # Open and prepare CSV file.
    with open(tablefile, 'w') as csvtobe:
        csvwriting = csv.DictWriter(csvtobe, fieldnames = columns)
        csvwriting.writeheader()
    
        # Loop on author keys in information dict.
        # and keep the associated value in "local" var.
        for author in info.keys():
            local = info[author]
            
            # Loop on all mentions of current author,
            # and combine with author information.
            try:
                for mention in mentions[author]:
                
                    csvwriting.writerow({
                        "Author": local["name"],
                        "Birth start": local["earliest-birth"],
                        "Birth stop" : local["latest-birth"],
                        "Death start" : local["earliest-death"],
                        "Death stop" : local["latest-death"],
                        "Witness" : mention[0],
                        "Part" : mention [1],
                        "Chapter" : mention [2],
                        "Section" : mention[3]
                    })
            except:
                print(author, "\n", local, "\n")
                continue

## FUNCTION: generating author CSV file.

In [40]:
def author_csv(dico, path):
    
    """
    Function taking a dictionary of author information and an
    export path and writing a CSV file for author timeline dataviz.
    
    :param dico: The dictionary of authors to show.
    :param path: The path to which the CSV file will be written.
    """
    
    # Open and prepare CSV file.
    with open(path, 'w') as csvtobe:
        csvwriting = csv.DictWriter(csvtobe, fieldnames = ["Author", "From", "To", "Param"])
        csvwriting.writeheader()
        
        # Loop on all authors in dictionary and store
        # current author information in "author" var.
        for entity in dico.keys():
            author = final[entity]
            
            # For all authors, make at least one line
            # containing the latest possible birth date
            # and earliest possible death date: these
            # represent the "sure" lifespan if there is one.
            
            csvwriting.writerow({
                "Author": author["name"],
                "From": author["latest-birth"],
                "To" : author["earliest-death"],
                "Param" : "Vie (certaine)",
            })
            
            # Then compare birth date columns and only
            # if dates are different, make another line
            # with this time span.
            
            if author["earliest-birth"] != author["latest-birth"]:
                
                csvwriting.writerow({
                    "Author": author["name"],
                    "From": author["earliest-birth"],
                    "To" : author["latest-birth"],
                    "Param" : "Naissance (imprécis)",
                })
            
            # Then compare death date columns and only
            # if dates are different, make another line
            # with this time span.
            
            if author["earliest-death"] != author["latest-death"]:
                
                csvwriting.writerow({
                    "Author": author["name"],
                    "From": author["earliest-death"],
                    "To" : author["latest-death"],
                    "Param" : "Mort (imprécis)",
                })

## Using the previously declared functions

In [46]:
# Initiate the root element for the XML debugging file.
listroot = ET.Element("people")

# Loop on witnesses: construct the path from initial vars,
# then trigger extract() function on current witness,
# so as to both make the according element for the XML debugging file,
# and fill the general author dictionary.

for witness in witnesses:
    fullpath = binpath + witness + einpath
    
    listroot.append(extract(witness, fullpath))

# Start a new dictionary for the sorted authors.
final = {}

# Fill the dictionary with only desired authors.
final = sort(authors)

# Write XML debugging file.
with open(checklist, "w") as failures:
    a_ecrire = ET.tostring(listroot, encoding="unicode", method="xml")
    failures.write(a_ecrire)

basnage


  0%|          | 0/360 [00:00<?, ?it/s]

berault


  0%|          | 0/145 [00:00<?, ?it/s]

merville


...

terrien


  0%|          | 0/92 [00:00<?, ?it/s]

Now sorting authors.


  0%|          | 0/489 [00:00<?, ?it/s]

{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Anian'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Johann Berthold Herold'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Arq.'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Goncanus'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Febur.'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': '\u2028Michel (saint)\u2028'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Masurier'}
{'earliest-birth': 'none', 'latest-birth': 'none', 'earliest-death': 'none', 'latest-death': 'none', 'name': 'Alexis Chartier'}
{'

In [47]:
# Start a general dictionary to store any mention information.
mentions = {}

# Once again loop on witnesses and construct path from initial vars,
# then trigger the get_refs() function to collect mention information.
for witness in witnesses:
    fullpath = binpath + witness + einpath
    get_refs(witness, fullpath, final.keys())

# Show the obtained dict. for checking.
print(mentions)

basnage


  0%|          | 0/4 [00:00<?, ?it/s]

berault


  0%|          | 0/6 [00:00<?, ?it/s]

merville


...

pesnelle


  0%|          | 0/3 [00:00<?, ?it/s]

terrien


  0%|          | 0/16 [00:00<?, ?it/s]

{'marculphe': [['basnage', '1', '1', '1'], ['basnage', '1', '1', '1'], ['basnage', '1', '1', '13'], ['basnage', '1', '1', '13'], ['basnage', '1', '1', '13'], ['basnage', '1', '9', '105'], ['basnage', '1', '9', '109'], ['basnage', '1', '9', '109'], ['basnage', '1', '9', '114'], ['basnage', '1', '9', '147'], ['basnage', '1', '9', '147'], ['basnage', '1', '11', '241'], ['basnage', '1', '11', '241'], ['basnage', '1', '11', '241'], ['basnage', '1', '11', '255'], ['basnage', '1', '11', '265'], ['basnage', '1', '11', '267'], ['basnage', '1', '13', '338'], ['basnage', '1', '15', '377'], ['basnage', '1', '15', '377'], ['basnage', '1', '15', '400'], ['basnage', '1', '15', '402'], ['pesnelle', '1', '2', '1'], ['pesnelle', '1', '2', '18'], ['pesnelle', '1', '10', '117'], ['pesnelle', '1', '12', '247'], ['pesnelle', '1', '12', '255'], ['pesnelle', '1', '12', '267'], ['pesnelle', '1', '12', '267'], ['pesnelle', '1', '18', '379'], ['pesnelle', '1', '18', '383'], ['pesnelle', '1', '18', '383'], ['pesn

In [48]:
# Write compilation CSV.
initial_csv(mentions, final)
author_csv(final, authortable)

appius-claudius 
 {'earliest-birth': '-0399', 'latest-birth': '-0300', 'earliest-death': '-0299', 'latest-death': '-0200', 'name': 'Appius Claudius Caecus'} 

